<a href="https://colab.research.google.com/github/cuberhaus/MD2/blob/main/src/test_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, ParameterGrid, train_test_split
from sklearn import metrics

import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier

# # import warnings filter
# from warnings import simplefilter
# # ignore all future warnings
# simplefilter(action='ignore', category=FutureWarning)

In [2]:
dataset_path = 'allUsers.lcl.csv'
# dataset_path = "../data/tommaso_data/allUsers.lcl.csv"

df = pd.read_csv(dataset_path)
#knn Cannot work with mining data, so replace them with a 0
df = df.replace('?', 0)
#df = df.dropna(thresh=len(df) * 0.10, axis=1)



In [3]:
def get_class_test_traingi_set(test_size_percent):
  df2 = df.drop(['User'], axis=1)
  df2 = df2.drop(['Class'], axis=1)
  labels_Class = df['Class']
  X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Class, test_size=test_size_percent, random_state=42)
  return X_train, X_test, y_train, y_test,




def get_user_test_traingi_set(test_size_percent):
  df2 = df.drop(['User'], axis=1)
  df2 = df2.drop(['Class'], axis=1)

  labels_Users = df['User']
  X_train, X_test, y_train, y_test,  = train_test_split(df2, labels_Users, test_size=test_size_percent, random_state=42)

  return X_train, X_test, y_train, y_test






# mvr function


In [4]:
from scipy.stats import mode
import numpy as np

def majority_vote_role(pre_trained_models_array, X_test, y_test):
  predictions = []
  for model in pre_trained_models_array:
    predictions.append(model.predict(X_test))
  
  predictions = np.array(predictions)
  final_prediction = (mode(predictions.T, axis=-1, keepdims=True)[0]).T[0]

  accuracy = 0
  for predicted_y, y in zip(final_prediction, y_test):

    if predicted_y == y :
      accuracy += 1

  return accuracy / len(y_test)




In [5]:
from sklearn.metrics import accuracy_score, f1_score

def save_results(clf, X_test, y_test, nclf, dataf):
    dataf.loc[nclf, 'test acc'] = accuracy_score(y_test, clf.predict(X_test))
    # dataf.loc[nclf, 'test f1 score (0)'] = f1_score(y_test, clf.predict(X_test), pos_label=0)
    # dataf.loc[nclf, 'test f1 score (1)'] = f1_score(y_test, clf.predict(X_test), pos_label=1)
    dataf.loc[nclf, 'test f1 score (W)'] = f1_score(y_test, clf.predict(X_test), average='macro')
    # df.loc[nclf,'ROC AUC'] = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
    return dataf
results_df = pd.DataFrame()



In [6]:
def test_model_and_save_result(model, name,  X_train, X_test, y_train, y_test, results_df):
  model.fit(X_train, y_train)
  results_df = save_results(model, X_test, y_test, name, results_df)

# KNN

In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)



In [8]:
X_train, X_test, y_train, y_test = get_class_test_traingi_set(0.20)
test_model_and_save_result(knn, "KNN", X_train, X_test, y_train, y_test, results_df)



In [9]:
results_df.sort_values(by=['test acc'], ascending=False)

,test acc,test f1 score (W)
KNN,0.959667,0.959219


# SVM

In [10]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler



In [11]:
#svm = SVC(kernel='rbf', gamma=0.0001, C = 10)
#test_model_and_save_result(svm, "SVM", X_train, X_test, y_train, y_test, results_df)

# Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [13]:
rf = RandomForestClassifier(max_depth=200, random_state=10)
test_model_and_save_result(rf, "Random Forest", X_train, X_test, y_train, y_test, results_df)

In [14]:
from sklearn.ensemble import AdaBoostClassifier

# Bayes

In [15]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
test_model_and_save_result(gnb, "GaussianNB", X_train, X_test, y_train, y_test, results_df)

# Desition tree

In [16]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text


decision_tree = DecisionTreeClassifier(random_state=2, max_depth=20)

test_model_and_save_result(decision_tree, "Decision_tree", X_train, X_test, y_train, y_test, results_df)

# AdaBoost

In [17]:
ABC = AdaBoostClassifier(n_estimators=200,
                         estimator = decision_tree,
                         learning_rate=1, 
                         random_state=23)
test_model_and_save_result(ABC, "AdaBoost", X_train, X_test, y_train, y_test, results_df)

## Show results:

In [18]:
results_df.sort_values(by=['test acc'], ascending=False)



,test acc,test f1 score (W)
AdaBoost,0.991997,0.991941
Random Forest,0.986876,0.986760
KNN,0.959667,0.959219
Decision_tree,0.958387,0.958062
GaussianNB,0.366965,0.248833


# Majority vore role class



> Blocco con rientro



In [19]:
results_df_class_mvr = pd.DataFrame()
X_train, X_test, y_train, y_test = get_class_test_traingi_set(0.20)

In [20]:
rf = RandomForestClassifier(max_depth=200, random_state=10)
test_model_and_save_result(rf, "Random Forest", X_train, X_test, y_train, y_test, results_df_class_mvr)

In [21]:
knn = KNeighborsClassifier(n_neighbors=1)
test_model_and_save_result(knn, "KNN", X_train, X_test, y_train, y_test, results_df_class_mvr)

In [22]:
svm = SVC(kernel='rbf', gamma=0.0001, C = 10)
test_model_and_save_result(svm, "SVM", X_train, X_test, y_train, y_test, results_df_class_mvr)

In [23]:
results_df_class_mvr.sort_values(by=['test acc'], ascending=False)

,test acc,test f1 score (W)
Random Forest,0.986876,0.986760
SVM,0.978553,0.978358
KNN,0.959667,0.959219


In [24]:

majority_vote_role([rf, svm, knn], X_test, y_test)

0.9838028169014085

# Cuser part

In [25]:

results_df_user = pd.DataFrame()
X_train, X_test, y_train, y_test = get_user_test_traingi_set(0.20)

In [26]:
#svm = SVC(kernel='rbf', gamma=0.0001, C = 10)
#test_model_and_save_result(svm, "SVM", X_train, X_test, y_train, y_test, results_df_user)

In [27]:
knn = KNeighborsClassifier(n_neighbors=1)
test_model_and_save_result(knn, "KNN", X_train, X_test, y_train, y_test, results_df_user)

In [28]:
rf = RandomForestClassifier(max_depth=200, random_state=10)
test_model_and_save_result(rf, "Random Forest", X_train, X_test, y_train, y_test, results_df_user)

In [29]:
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=20)
test_model_and_save_result(decision_tree, "Decision_tree", X_train, X_test, y_train, y_test, results_df_user)

In [30]:
gnb = GaussianNB()
test_model_and_save_result(gnb, "GaussianNB", X_train, X_test, y_train, y_test, results_df_user)

In [31]:
ABC = AdaBoostClassifier(n_estimators=200,
                         estimator = decision_tree,
                         learning_rate=1, 
                         random_state=23)
test_model_and_save_result(ABC, "AdaBoost", X_train, X_test, y_train, y_test, results_df_user)

In [32]:
results_df_user.sort_values(by=['test acc'], ascending=False)


,test acc,test f1 score (W)
AdaBoost,0.989757,0.950960
Random Forest,0.983931,0.930375
Decision_tree,0.900960,0.833709
KNN,0.864661,0.796469
GaussianNB,0.121447,0.090120


# Majority vore role user



> Blocco con rientro



In [33]:
results_df_user_mvr = pd.DataFrame()

X_train, X_test, y_train, y_test = get_user_test_traingi_set(0.20)

rf1 = RandomForestClassifier(max_depth=200, random_state=10)
test_model_and_save_result(rf1, "Random Forest1", X_train[:len(y_train)//2], X_test, y_train[:len(y_train)//2], y_test, results_df_user_mvr)

rf2 = RandomForestClassifier(max_depth=100, random_state=25)
test_model_and_save_result(rf2, "Random Forest2", X_train[len(y_train)//2:], X_test, y_train[len(y_train)//2:], y_test, results_df_user_mvr)

knn = KNeighborsClassifier(n_neighbors=1)
test_model_and_save_result(knn, "KNN", X_train, X_test, y_train, y_test, results_df_user_mvr)

svm = SVC(kernel='rbf', gamma=0.0001, C = 10)
test_model_and_save_result(svm, "SVM", X_train, X_test, y_train, y_test, results_df_user_mvr)



In [34]:
results_df_user_mvr.sort_values(by=['test acc'], ascending=False)

,test acc,test f1 score (W)
Random Forest2,0.970551,0.895484
Random Forest1,0.970294,0.889484
SVM,0.940589,0.888089
KNN,0.864661,0.796469


In [35]:

majority_vote_role([rf1, rf2, svm, knn], X_test, y_test)

0.9660691421254801